In [2]:
import torch
from numpy.random import multivariate_normal as mn 

In [3]:
torch.__version__

'1.12.1'

In [4]:
# import seaborn as sns
# import torch

data_1 = torch.Tensor(mn([1, 0], [[.01, 0], [0, .01]], 1000), dtype=torch.float)
data_2 = torch.Tensor(mn([0, 1], [[.01, 0], [0, .01]], 1000), dtype=torch.float)

data = torch.cat([data_1, data_2])

# sns.plot(data[:,0], data[:,1])
# plt.show()

: 

: 

In [ ]:
class DictionaryLearning(torch.nn.Module):

    def __init__(self, dim, n_terms):
        super(DictionaryLearning, self).__init__()
        self.dim = dim
        self.n_terms = n_terms
        self.dictionary = torch.nn.Parameter(torch.rand((n_terms, dim),
                                    dtype=torch.float,
                                    requires_grad=True
                                    ))

        self.attributor = self.build_attributor()


    def build_attributor(self):
        layers = torch.nn.ModuleList()
        layers.append(torch.nn.Linear(self.dim, self.n_terms))
        layers.append(torch.nn.Softmax())
        return torch.nn.Sequential(*layers)

    def forward(self, x):
        return self.attributor(x)

    def loss(self, x, attributions):
        reconstruct = torch.matmul(attributions, self.dictionary)
        return torch.nn.MSELoss()(x, reconstruct)
    

In [ ]:
epochs = 10000
n_terms = 2
dim = 2

model = DictionaryLearning(dim, n_terms)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for e in range(epochs):
    attributions = model(data)
    loss = model.loss(data, attributions)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()